# Experiment 2

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np

from vgc_project.r import create_R_model_interface, ImmutableDataFrame
from vgc_project.parameter_fit import create_fit_vgc_model_to_trials, Trial

joblib_cache_location = "./_analysiscache"
rmods = create_R_model_interface(joblib_cache_location=joblib_cache_location)
fit_vgc_model_to_trials = create_fit_vgc_model_to_trials(joblib_cache_location=joblib_cache_location)

import analysisutils
from analysisutils import predictor_names, short_predictor_names
from prep_data import \
    mazes,\
    model_preds,\
    get_exp2_at

exp2_at = get_exp2_at()

R[write to console]: Loading required package: Matrix



In [2]:
exp2_early_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'earlyterm'])
exp2_full_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'full'])
exp2_at_im = ImmutableDataFrame(exp2_at)

## HGLM with VGC

In [3]:
exp2_full_vgc_sum = analysisutils.single_predictor_analysis(
    name='Exp. 2 VGC full model',
    data=exp2_full_at_im,
    dv='attention_N',
    model_func='lmer',
    random_effects='(1 | sessionId) + (1 | grid)',
    predictor='vgc_weight_Z',
    rmods=rmods,
    coeff_digits=3,
    normalized_predictor=True
)
exp2_early_vgc_sum = analysisutils.single_predictor_analysis(
    name='Exp. 2 VGC early model',
    data=exp2_early_at_im,
    dv='attention_N',
    model_func='lmer',
    random_effects='(1 | sessionId) + (1 | grid)',
    predictor='vgc_weight_Z',
    rmods=rmods,
    coeff_digits=3,
    normalized_predictor=True
)

with open("./inputs/exp2_early_vgc_single_summary.tex", 'w') as file:
    file.write(exp2_early_vgc_sum.summary)
print(exp2_early_vgc_sum.summary)
with open("./inputs/exp2_full_vgc_single_summary.tex", 'w') as file:
    file.write(exp2_full_vgc_sum.summary)
print(exp2_full_vgc_sum.summary)

$\chi^2(1) = 679.20, p  < 1.0 \times 10^{-16}$; $\beta = 0.106$, S.E. $= 0.004$
$\chi^2(1) = 726.95, p  < 1.0 \times 10^{-16}$; $\beta = 0.115$, S.E. $= 0.004$


## Interaction analysis

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
library(lme4)
library(lmerTest)
library(lmtest)
library(scales)

Attaching package: ‘lmerTest’



    lmer



    step



Attaching package: ‘zoo’



    as.Date, as.Date.numeric




In [5]:
%%R -i exp2_at_im
exp2_at_im$earlyterm <- factor(exp2_at_im$earlyterm)
contrasts(exp2_at_im$earlyterm) <- contr.sum(2)
print(contrasts(exp2_at_im$earlyterm))
onlymain <- lmer(
    attention_N ~
                vgc_weight_Z
                    + graph_based_hitcount_Z
                    + log_traj_based_hitcount_Z
                    + optpolicy_dist_Z
                    + goal_dist_Z
                    + start_dist_Z
                    + walls_dist_Z
                    + center_dist_Z
                    + bottleneck_dist_Z
                    + sr_occ_Z
                + earlyterm
                + (1 | sessionId) + (1 | grid),
    data=exp2_at_im,
    control=lmerControl(optimizer="bobyqa", optCtrl=list(maxfun=1e+05)),
    REML=F
)
summary(onlymain)

          [,1]
earlyterm    1
full        -1
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: 
attention_N ~ vgc_weight_Z + graph_based_hitcount_Z + log_traj_based_hitcount_Z +  
    optpolicy_dist_Z + goal_dist_Z + start_dist_Z + walls_dist_Z +  
    center_dist_Z + bottleneck_dist_Z + sr_occ_Z + earlyterm +  
    (1 | sessionId) + (1 | grid)
   Data: exp2_at_im
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 1e+05))

     AIC      BIC   logLik deviance df.resid 
  5173.9   5286.3  -2571.9   5143.9    13306 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1959 -0.7532 -0.0261  0.7669  3.3219 

Random effects:
 Groups    Name        Variance Std.Dev.
 sessionId (Intercept) 0.019764 0.14058 
 grid      (Intercept) 0.001595 0.03994 
 Residual              0.082815 0.28778 
Number of obs: 13321, groups:  sessionId, 162; grid, 12

Fixed effects:
                            Estimate Std. Error  

In [6]:
%%R -o interactions_res
interactions_res <- rbind(
    anova(
        onlymain,
        update(onlymain, ~ . + earlyterm:vgc_weight_Z)
    ),
    anova(
        onlymain,
        update(onlymain, ~ . + earlyterm:optpolicy_dist_Z)
    ),
    anova(
        onlymain,
        update(onlymain, ~ . + earlyterm:log_traj_based_hitcount_Z)
    ),
    anova(
        onlymain,
        update(onlymain, ~ . + earlyterm:graph_based_hitcount_Z)
    )
)

In [7]:
%%R -o optpolicy_interaction_maineff_sum,optpolicy_interaction_interaction_sum,log_traj_interaction_maineff_sum,log_traj_interaction_interaction_sum
# Optimal Policy Distance Interaction Model
optpolicy_dist_res <- summary(update(onlymain, ~ . + earlyterm:optpolicy_dist_Z))
optpolicy_interaction_maineff_sum <- paste(
    "$\\beta = ", round(optpolicy_dist_res$coefficients["optpolicy_dist_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(optpolicy_dist_res$coefficients["optpolicy_dist_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
optpolicy_interaction_interaction_sum <- paste(
    "$\\beta = ", round(optpolicy_dist_res$coefficients["optpolicy_dist_Z:earlyterm1",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(optpolicy_dist_res$coefficients["optpolicy_dist_Z:earlyterm1",]["Std. Error"], digits=3),
    "$", sep=""
)
# Trajectory-based Heuristic Search Interaction Model
log_traj_res <- summary(update(onlymain, ~ . + earlyterm:log_traj_based_hitcount_Z))
log_traj_interaction_maineff_sum <- paste(
    "$\\beta = ", round(log_traj_res$coefficients["log_traj_based_hitcount_Z",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(log_traj_res$coefficients["log_traj_based_hitcount_Z",]["Std. Error"], digits=3),
    "$", sep=""
)
log_traj_interaction_interaction_sum <- paste(
    "$\\beta = ", round(log_traj_res$coefficients["log_traj_based_hitcount_Z:earlyterm1",]["Estimate"], digits=3),
    ", \\text{S.E.} = ", round(log_traj_res$coefficients["log_traj_based_hitcount_Z:earlyterm1",]["Std. Error"], digits=3),
    "$", sep=""
)

In [8]:
optpolicy_interaction_maineff_sum = optpolicy_interaction_maineff_sum[0]
optpolicy_interaction_interaction_sum = optpolicy_interaction_interaction_sum[0]
log_traj_interaction_maineff_sum = log_traj_interaction_maineff_sum[0]
log_traj_interaction_interaction_sum = log_traj_interaction_interaction_sum[0]

In [9]:
optpolicy_interaction_maineff_sum , optpolicy_interaction_interaction_sum, log_traj_interaction_maineff_sum, log_traj_interaction_interaction_sum

('$\\beta = -0.082, \\text{S.E.} = 0.004$',
 '$\\beta = 0.009, \\text{S.E.} = 0.003$',
 '$\\beta = -0.033, \\text{S.E.} = 0.006$',
 '$\\beta = 0.013, \\text{S.E.} = 0.003$')

In [10]:
with open("./inputs/exp2_optpolicy_interaction_maineff_sum.tex", 'w') as file:
    file.write(optpolicy_interaction_maineff_sum)
with open("./inputs/exp2_optpolicy_interaction_interaction_sum.tex", 'w') as file:
    file.write(optpolicy_interaction_interaction_sum)
with open("./inputs/exp2_log_traj_interaction_maineff_sum.tex", 'w') as file:
    file.write(log_traj_interaction_maineff_sum)
with open("./inputs/exp2_log_traj_interaction_interaction_sum.tex", 'w') as file:
    file.write(log_traj_interaction_interaction_sum)

In [11]:
exp2_interactions_res = interactions_res[['update' in i for i in interactions_res.index]].copy()
exp2_interactions_res.index = [i.replace("update(onlymain, ~. + earlyterm:", "").replace(")", "")
                               for i in exp2_interactions_res.index]
exp2_interactions_res["Bonferroni Pr(>Chisq)"] = exp2_interactions_res['Pr(>Chisq)'].apply(lambda p: min(p*4, 1.0))

In [12]:
for pred, row in exp2_interactions_res.iterrows():
    pred_interaction_summary = \
        f"$\chi^2({int(row['Chi Df']):d}) = " + \
        f"{row['Chisq']:.2f}" + \
        ", p " + analysisutils.pval_to_string(row["Bonferroni Pr(>Chisq)"]) + "$"
    pred = pred.replace("_Z", "")
    filename = f"./inputs/exp2_{pred}_interaction_lesion_llr.tex"
    print(filename)
    with open(filename, 'w') as file:
        file.write(pred_interaction_summary)
    print(pred, ": ", pred_interaction_summary)
    print()
    # print(pred, row)

./inputs/exp2_vgc_weight_interaction_lesion_llr.tex
vgc_weight :  $\chi^2(1) = 1.01, p = 1.0$

./inputs/exp2_optpolicy_dist_interaction_lesion_llr.tex
optpolicy_dist :  $\chi^2(1) = 12.15, p = 0.0020$

./inputs/exp2_log_traj_based_hitcount_interaction_lesion_llr.tex
log_traj_based_hitcount :  $\chi^2(1) = 24.75, p = 2.6 \times 10^{-6}$

./inputs/exp2_graph_based_hitcount_interaction_lesion_llr.tex
graph_based_hitcount :  $\chi^2(1) = 1.67, p = 0.79$

